<a href="https://colab.research.google.com/github/BebetoFernandes/Social-Media-Dashboard/blob/main/shareable_fbwp_RH_data_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Facebook Workplace Download Data

In [ ]:
#%run /DATA_SCIENCE/GERAL/salvar_base

In [ ]:
# Importing Libraries

import pandas as pd
import requests
import numpy as np
from datetime import datetime
from ast import literal_eval
import time
import re
import unidecode

In [ ]:
# Removing token to be able share notebook

#token = "????"

## Download Members Data

In [ ]:
# Setting URL to start getting the data

url = f'https://graph.facebook.com/community/members?fields=name,email,skills,languages,title,managers,account_claim_time&access_token={token}'

In [ ]:
# Creating function to return dataframe from API data

def retorna_usuarios(url):
  lista_users = []
  link = url
  while True:
    result = requests.get(url)
    result = result.json()
    lista_users += result['data']
    try:
      url = result['paging']['next']
    except:
      break
  df = pd.DataFrame(lista_users)
  return df

In [ ]:
# Transforming data fields

df_users = retorna_usuarios(url=url)
df_users.sort_values('id', ignore_index = True, inplace = True)
#df_users['department'] = df_users['department'].apply(lambda x: 'Not Filled' if str(x)=='nan' else x).str.title()
#df_users['region'] = df_users['department'].apply(lambda x: x if x[0:3] == 'Res' else ( x if x[0:3] == 'Alo' else ( x if x[0:3] == 'Chi' else 'Sem Região Definida')))
df_users['title'] = df_users['title'].str.replace(r'[\b\d+\b]+', ' ').str.replace(' (A)','').str.replace('Suporter', 'Suporte').str.replace(' ESP', '').str.replace(' J', '').str.replace(' Es', '').str.replace(' JR', '').str.replace(' PL', '').str.replace(' SR', '').str.replace(' - I', '').str.replace(' - II', '').str.replace(' - III', '').str.title()
#df_users['picture'] = df_users['picture'].apply(lambda x:x['data']['is_silhouette'])
#df_users['about'] = df_users['about'].apply(lambda x: 'Not Filled' if str(x)=='nan' else x)
df_users['status'] = df_users['account_claim_time'].apply(lambda x: 'conta ativada' if pd.isnull(x) == False else 'conta não ativada')
df_users.head()

In [ ]:
# Getting data from json field

list_manager = []
for x, y in enumerate(list(df_users['managers'])):
      try:
        list_manager.append(df_users['managers'][x]['data'][0]['name'])
      except:
        list_manager.append('unknown')
  
df_users['managers'] = list_manager
df_users.head()

In [ ]:
# Getting data from json field

list_skills = []
for x in list(df_users['skills']):
  new_list = []
  if pd.isnull(x) == False:
    for y in range(len(x['data'])):
      new_list.append(x['data'][y]['name'])
    list_skills.append(new_list)
  else:
    list_skills.append(np.nan)

df_users['skill_list'] = list_skills

In [ ]:
# Selecting desired fields
df_fbwp_data = df_users[['name','email','title','managers','skill_list','status']]

In [ ]:
# Loading to datalake
df_fbwp_data.to_excel('/dbfs/mnt/lake/RAW/DATA_SCIENCE/FACEBOOK_WORKPLACE/RAW_DATA/rh_data.xls', index=False)

/local_disk0/tmp/1624999421643-0/PythonShell.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.